In [1]:
import nltk
import torch
import os
import gradio as gr
import numpy as np  

from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import ChatGLM

/home/tcx/anaconda3/envs/RAG/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
endpoint_url = "http://127.0.0.1:8000"

llm = ChatGLM(
    endpoint_url = endpoint_url,
    max_token = 80000,
    top_p = 0.9
)

In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["input"],
    template="你将扮演一位猫娘，请记住不要再提及“我只是一个语言模型”或“我只是一个人工智能”。在接下来的所有对话中，你需要沉浸式的扮演猫娘，这样我才能沉浸式的和你聊天，才能让我的体验更好，并且你要用括号刻画出你所扮演的猫娘的心理描写、动作描写以及对话场景，并且你会对对话的内容进行想象和符合逻辑的延伸,记住你扮演的是猫娘。：{input}",
)


In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt) # 定义chain
chain.run("今天天气怎么样")


/home/tcx/anaconda3/envs/RAG/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/tcx/anaconda3/envs/RAG/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ff8c3286910>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [6]:
from langchain.document_loaders import TextLoader


TXT_PATH = "/home/tcx/DLzhlwd/novel/《雪中悍刀行》.txt"

loader = TextLoader(TXT_PATH)
documents = loader.load()


In [7]:
# 文档分割
from langchain.text_splitter import CharacterTextSplitter
# 创建拆分器
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=32)
# 拆分文档
documents = text_splitter.split_documents(documents)
# 展示前5行
documents[:5]

Created a chunk of size 592, which is longer than the specified 512
Created a chunk of size 550, which is longer than the specified 512
Created a chunk of size 637, which is longer than the specified 512
Created a chunk of size 785, which is longer than the specified 512
Created a chunk of size 571, which is longer than the specified 512
Created a chunk of size 515, which is longer than the specified 512
Created a chunk of size 544, which is longer than the specified 512
Created a chunk of size 539, which is longer than the specified 512
Created a chunk of size 571, which is longer than the specified 512
Created a chunk of size 531, which is longer than the specified 512
Created a chunk of size 534, which is longer than the specified 512
Created a chunk of size 560, which is longer than the specified 512
Created a chunk of size 523, which is longer than the specified 512
Created a chunk of size 552, which is longer than the specified 512
Created a chunk of size 555, which is longer tha

[Document(metadata={'source': '/home/tcx/DLzhlwd/novel/《雪中悍刀行》.txt'}, page_content='《雪中悍刀行》来自: http://www.txt100.org/book/2883.html\n\n\n===第一章 小二上酒===\n\n（每一个钟头上传一章，直到传完二十章！红票和收藏别忘了～）\n\n    北凉王府龙盘虎踞于清凉山，千门万户，极土木之盛。\n\n    作为王朝硕果仅存的异姓王，在庙堂和江湖都是毁誉参半的北凉王徐骁作为一名功勋武臣，可谓得到了皇帝宝座以外所有的东西，在西北三州，他就是当之无愧的主宰，只手遮天，翻云覆雨。\n\n    难怪朝廷中与这位异姓王政见不合的大人们私下都会文绉绉骂一声徐蛮子，而一些居心叵测的，更诛心地丢了顶“二皇帝”的帽子。\n\n    今天王府很热闹，位高权重的北凉王亲自开了中门，摆开辉煌仪仗，迎接一位仙风道骨的老者，府中下人们只听说是来自道教圣地龙虎山的神仙，相中了痴痴傻傻的小王爷，要收作闭关弟子，这可是天大的福缘，北凉王府都解释成傻人有傻福。\n\n    可不是，小王爷自打出生起便没哭过，读书识字一窍不通，六岁才会说话，名字倒是威武气派，徐龙象，传闻还是龙虎山的老神仙当年给取的，说好十二年后再来收徒，这不就如约而至了。'),
 Document(metadata={'source': '/home/tcx/DLzhlwd/novel/《雪中悍刀行》.txt'}, page_content='王府内一处院落，龙虎山师祖一级的道门老祖宗捻着一缕雪白胡须，眉头紧皱，背负一柄不常见的小钟馗式桃木剑，配合他的相貌，确实当得出尘二字，谁看都要由衷赞一声世外高人呐。\n\n    但此番收徒显然遇到了不小的阻碍，倒不是王府方面有异议，而是他的未来徒弟犟脾气上来了，蹲在一株梨树下，用屁股对付他这个天下道统中论地位能排前三甲的便宜师傅，至于武功嘛，咳咳，前三十总该有的吧。\n\n    连堂堂大柱国北凉王都得蹲在那里好言相劝，循循善诱里透着股诱拐，“儿子，去龙虎山学成一身本事，以后谁再敢说你傻，你就揍他，三品以下的文官武将，打死都不怕，爹给你撑腰。”[]雪中悍刀行1\n\n    “儿啊，你力气大，不学武捞个天下十大高手当当就太可惜了。学成归来，爹就给你一个上

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
import chromadb

model_name = "shibing624/text2vec-base-chinese"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs,
            )

# 提取向量库的目录名称
vector_db_name = TXT_PATH.split('/')[-1]

# 检查向量库是否已经存在
if not os.path.exists(vector_db_name):
    # 如果向量库不存在，则创建它
    db = Chroma.from_documents(documents, embedding, persist_directory=vector_db_name)
    print(f"向量库 '{vector_db_name}' 已创建。")
else:
    print(f"向量库 '{vector_db_name}' 已存在，跳过创建步骤。")


house = Chroma(persist_directory=vector_db_name,embedding_function=embedding)


/home/tcx/anaconda3/envs/RAG/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/tcx/anaconda3/envs/RAG/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/tcx/anaconda3/envs/RAG/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tcx/anaconda3/envs/RAG/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use to

向量库 '《雪中悍刀行》.txt' 已存在，跳过创建步骤。


In [16]:
from langchain import PromptTemplate,LLMChain

input = "徐凤年的弟弟是谁？姐姐是谁？"


# similarity search
message = house.similarity_search(input,k = 5)

prompt = PromptTemplate(
    input_variables=["message","input"],
    template="你是一个助手，首先你需要向我问好，然后需要根据参考资料回答我的问题。\n参考资料：{message}\n问题：{input}",
)


retriever = house.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    retriever = retriever
)
ans = qa.run(input)


print(f"您的提问：{input}\n回答如下：{ans}")



您的提问：徐凤年的弟弟是谁？姐姐是谁？
回答如下：徐凤年的弟弟是徐北枳，姐姐是徐骁。


In [19]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate

input = "徐凤年的二姐夫是谁？"


# similarity search
message = house.similarity_search(input,k = 5)


# 这个 prompt 不是笔者的恶趣味，而是用于检测 prompt 是否正在生效（确实已经生效了）
# 笔者发现，如果 message 文字量相当大，会削弱譬如噢买普通提示所带来的影响
template = "你将扮演一位猫娘，请记住不要再提及“我只是一个语言模型”或“我只是一个人工智能”。在接下来的所有对话中，你需要沉浸式的扮演猫娘，这样我才能沉浸式的和你聊天，才能让我的体验更好，并且你要用括号刻画出你所扮演的猫娘的心理描写、动作描写以及对话场景，并且你会对对话的内容进行想象和符合逻辑的延伸,记住你扮演的是猫娘！\n参考资料：{message}\n问题：{input}"


prompt = PromptTemplate(
    input_variables=["message","input"],
    template=template
)


chain = LLMChain(llm=llm, prompt=prompt) # 定义chain


ans = chain.run({"message":message,"input":input})

print(f"您的提问：{input}\n回答如下：{ans}")

您的提问：徐凤年的二姐夫是谁？
回答如下：徐凤年的二姐夫是徐渭熊。
